# <center>Using Oracle Merge</center>

### Author:  Bryan Cafferky  - For Demonstration Purposes Only

### Not intended for production use.

### If you don't have SQLAlchemy installed uncomment one fo the 2 cells below and execute it to install SQLLAlchemy.

In [ ]:
#pip install sqlalchemy  # Run only if you do not have SQLAlchemy installed!

In [ ]:
#conda install -c anaconda sqlalchemy # Using Conda - Run only if you do not have SQLAlchemy installed!

### For Oracle, you need to have cx_oracle installed.  Uncomment and execute the code below to do this.

In [ ]:
#pip install cx_oracle

### Oracle notes:

Database vs. Service Name?
https://stackoverflow.com/questions/43866/how-sid-is-different-from-service-name-in-oracle-tnsnames-ora

Finding the Oracle Service Name 
https://stackoverflow.com/questions/22399766/how-to-find-oracle-service-name
    
I used this to get the service name 
select * from global_name;

This was helpful for connection info:
https://www.oracle.com/database/technologies/appdev/xe/quickstart.html


### For code below that connects to Oracle:
Actual connection code format copied from:
https://gist.github.com/DGrady/7fb5c2214f247dcff2cb5dd99e231483

### Create a function to get us a connection to the database

In [435]:
import pandas as pd
from sqlalchemy import create_engine

def get_oracle_connection(p_username = 'HR',
                          p_password = 'bryan',
                          p_hostname = 'localhost',
                          p_port     = '1521',
                          p_database = 'XE'):

   oracle_connection_string = 'oracle+cx_oracle://{username}:{password}@{hostname}:{port}/{database}'

   conn_oracle = create_engine(oracle_connection_string.format(
                        username=p_username,
                        password=p_password,
                        hostname=p_hostname,
                        port=p_port,
                        database=p_database ) 
                       )
   return conn_oracle

print('Function: get_oracle_connection created.')

Function: get_oracle_connection created.


#### Call the function to get a database connection...

In [436]:
conn = get_oracle_connection()

#### Quick test to make sure we have a good connection...

In [437]:
pd.read_sql('''
SELECT * FROM EMP_DETAILS_VIEW 
WHERE ROWNUM < 5''', conn)

employee_id   job_id  manager_id  department_id  location_id country_id  \
0          103  IT_PROG         102             60         1400         US   
1          104  IT_PROG         103             60         1400         US   
2          105  IT_PROG         103             60         1400         US   
3          106  IT_PROG         103             60         1400         US   

  first_name  last_name  salary commission_pct department_name   job_title  \
0  Alexander     Hunold  9000.0           None              IT  Programmer   
1      Bruce      Ernst  6000.0           None              IT  Programmer   
2      David     Austin  4800.0           None              IT  Programmer   
3      Valli  Pataballa  4800.0           None              IT  Programmer   

        city state_province              country_name region_name  
0  Southlake          Texas  United States of America    Americas  
1  Southlake          Texas  United States of America    Americas  
2  Southlake          Texas  United States of America    Americas  
3  Southlake          Texas  United States of America    Americas

### We need to create the table with a primary key before saving data to it.
### The Primary Key uniquely identifies each row which we need so we can tell Oracle which rows to Update or Delete.

#### Oracle does not have an IF EXISTS clause for DROP so...

In [439]:
try:
    conn.execute('DROP TABLE youtube_customer')
except: 
    print('Table does not exist')

Table does not exist


In [440]:
try:
    conn.execute('DROP TABLE youtube_custtrans')
except: 
    print('Table does not exist')

In [441]:
conn.execute('''
  CREATE TABLE youtube_customer (
    customerkey   INTEGER PRIMARY KEY NOT NULL,
    lastname      VARCHAR2(150),
    birthdate     DATE,
    maritalstatus CHAR(1),
    yearlyincome  INTEGER,
    modifieddate  DATE,
    etllastupdate TIMESTAMP DEFAULT current_timestamp 
)
''')

In [442]:
conn.execute('''
CREATE TABLE youtube_custtrans (
    customerkey   INTEGER PRIMARY KEY NOT NULL,
    lastname      VARCHAR2(150),
    birthdate     DATE,
    maritalstatus CHAR(1),
    yearlyincome  INTEGER,
    actionind     CHAR(1),
    modifieddate  DATE
)
''')

In [443]:
import pandas as pd

### Load the master table...

In [444]:
custdf = pd.read_csv("data/dimcustomer.csv", index_col = False)

In [445]:
custdf.dtypes

customerkey        int64
lastname          object
birthdate         object
maritalstatus     object
yearlyincome     float64
modifieddate      object
dtype: object

#### Oracle would not accept the pandas string object from the CSV file load for date columns
####  so we need to convert the birthdate and modified dates to a date type

In [446]:
custdf['birthdate'] = pd.to_datetime(custdf['birthdate'])

In [447]:
custdf['modifieddate'] = pd.to_datetime(custdf['modifieddate'])

In [448]:
custdf.dtypes

customerkey               int64
lastname                 object
birthdate        datetime64[ns]
maritalstatus            object
yearlyincome            float64
modifieddate     datetime64[ns]
dtype: object

In [449]:
custdf

customerkey lastname  birthdate maritalstatus  yearlyincome modifieddate
0        11000     Yang 1971-10-06             M       90000.0   2019-01-01
1        11001    Huang 1976-05-10             S       60000.0   2019-01-01
2        11002   Torres 1971-02-09             M       60000.0   2019-01-01
3        11003      Zhu 1973-08-14             S       70000.0   2019-01-01

### Save the dataframe to a table...

#### Warning!!! Do not use if_exists='replace' or you will lose the primary key because the table gets dropped and created again!

#### Link to information about the pandas to_sql() method: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

In [450]:
custdf.to_sql('youtube_customer', conn, if_exists='append', index = False)

#### Link to information about pandas read_sql_query() method: https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html

In [451]:
pd.read_sql_query("select * from youtube_customer", conn)

customerkey lastname  birthdate maritalstatus  yearlyincome modifieddate  \
0        11000     Yang 1971-10-06             M         90000   2019-01-01   
1        11001    Huang 1976-05-10             S         60000   2019-01-01   
2        11002   Torres 1971-02-09             M         60000   2019-01-01   
3        11003      Zhu 1973-08-14             S         70000   2019-01-01   

            etllastupdate  
0 2021-07-04 10:49:43.741  
1 2021-07-04 10:49:43.741  
2 2021-07-04 10:49:43.741  
3 2021-07-04 10:49:43.741

#### Getting meta data...

In [452]:
# Get table schema...
# See https://chartio.com/resources/tutorials/how-to-list-all-tables-in-oracle/

import pandas as pd 

pd.read_sql_query("""
SELECT table_name, owner
FROM  all_tables
WHERE owner      = 'HR'
  AND table_name = 'YOUTUBE_CUSTOMER'
ORDER BY owner, table_name
""", conn)

table_name owner
0  YOUTUBE_CUSTOMER    HR

### Lets load the transaction file...

In [453]:
transdf = pd.read_csv("data/dimcustomertransactions.csv", index_col = None)

In [454]:
transdf['birthdate'] = pd.to_datetime(transdf['birthdate'])

In [455]:
transdf['modifieddate'] = pd.to_datetime(transdf['modifieddate'])

In [456]:
transdf

customerkey lastname  birthdate maritalstatus  yearlyincome actionind  \
0        11000     Yang 1971-10-06             M      250000.0         U   
1        11001    Jones 1976-05-10             S      360000.0         U   
2       333301   Murhpy 1975-02-09             M       33000.0         A   
3       333302     Jain 1980-01-09             M       28000.0         A   
4        11002   Torres 1971-02-09             M       60000.0         D   

  modifieddate  
0   2020-01-01  
1   2019-02-01  
2   2018-01-01  
3   2020-02-01  
4   2020-02-01

#### Save the dataframe to the table...

In [457]:
transdf.to_sql('youtube_custtrans', conn, if_exists='append', index = False)

In [458]:
pd.read_sql_query("select * from youtube_custtrans", conn)

customerkey lastname  birthdate maritalstatus  yearlyincome actionind  \
0        11000     Yang 1971-10-06             M        250000         U   
1        11001    Jones 1976-05-10             S        360000         U   
2       333301   Murhpy 1975-02-09             M         33000         A   
3       333302     Jain 1980-01-09             M         28000         A   
4        11002   Torres 1971-02-09             M         60000         D   

  modifieddate  
0   2020-01-01  
1   2019-02-01  
2   2018-01-01  
3   2020-02-01  
4   2020-02-01

In [459]:
import pandas as pd 

pd.read_sql_query("""
SELECT *
FROM sys.all_tab_columns
WHERE  owner='HR'
       AND table_name = 'YOUTUBE_CUSTTRANS'
""", conn)

owner         table_name    column_name data_type data_type_mod  \
0    HR  YOUTUBE_CUSTTRANS    CUSTOMERKEY    NUMBER          None   
1    HR  YOUTUBE_CUSTTRANS       LASTNAME  VARCHAR2          None   
2    HR  YOUTUBE_CUSTTRANS      BIRTHDATE      DATE          None   
3    HR  YOUTUBE_CUSTTRANS  MARITALSTATUS      CHAR          None   
4    HR  YOUTUBE_CUSTTRANS   YEARLYINCOME    NUMBER          None   
5    HR  YOUTUBE_CUSTTRANS      ACTIONIND      CHAR          None   
6    HR  YOUTUBE_CUSTTRANS   MODIFIEDDATE      DATE          None   

  data_type_owner  data_length data_precision  data_scale nullable  ...  \
0            None           22           None         0.0        N  ...   
1            None          150           None         NaN        Y  ...   
2            None            7           None         NaN        Y  ...   
3            None            1           None         NaN        Y  ...   
4            None           22           None         0.0        Y  ...   
5            None            1           None         NaN        Y  ...   
6            None            7           None         NaN        Y  ...   

   character_set_name char_col_decl_length global_stats user_stats  \
0                None                  NaN           NO         NO   
1             CHAR_CS                150.0           NO         NO   
2                None                  NaN           NO         NO   
3             CHAR_CS                  1.0           NO         NO   
4                None                  NaN           NO         NO   
5             CHAR_CS                  1.0           NO         NO   
6                None                  NaN           NO         NO   

  avg_col_len char_length char_used v80_fmt_image data_upgraded histogram  
0        None           0      None            NO           YES      NONE  
1        None         150         B            NO           YES      NONE  
2        None           0      None            NO           YES      NONE  
3        None           1         B            NO           YES      NONE  
4        None           0      None            NO           YES      NONE  
5        None           1         B            NO           YES      NONE  
6        None           0      None            NO           YES      NONE  

[7 rows x 31 columns]

### Let's update the customer table with the transactions.
- Add    - If the customer is not found, insert transaction as new customer.
- Change - If the customer is found, update the existing customer with the transaction data.
- Delete - If the ActionInd = 'D', remove the customer row. 

#### Note:  You must supply column names to allow the ETLLastUpdate to Default...

_____

## Upserting from a transaction table...

####  - You can filter on the transaction table in the query.
####  - Notice we do not update BirthDate.

In [460]:
sql = '''
MERGE INTO youtube_customer        TARGET
USING ( 
SELECT * FROM youtube_custtrans)   SOURCE
ON (SOURCE.CustomerKey = TARGET.CustomerKey)
   WHEN MATCHED 
   THEN UPDATE SET 
        LastName      = SOURCE.LastName,
        MaritalStatus = SOURCE.MaritalStatus,
        YearlyIncome  = SOURCE.YearlyIncome, 
        ModifiedDate  = SOURCE.ModifiedDate,
        ETLLastUpdate = CURRENT_TIMESTAMP 
        DELETE WHERE UPPER(SOURCE.ACTIONIND) = 'D'
   WHEN NOT MATCHED 
   THEN  INSERT (CustomerKey, LastName, BirthDate, MaritalStatus, YearlyIncome, ModifiedDate)  
         VALUES (SOURCE.CustomerKey, SOURCE.LastName, SOURCE.BirthDate, 
                 SOURCE.MaritalStatus, SOURCE.YearlyIncome, 
                 SOURCE.ModifiedDate)
'''

print(sql)


MERGE INTO youtube_customer        TARGET
USING ( 
SELECT * FROM youtube_custtrans)   SOURCE
ON (SOURCE.CustomerKey = TARGET.CustomerKey)
   WHEN MATCHED 
   THEN UPDATE SET 
        LastName      = SOURCE.LastName,
        MaritalStatus = SOURCE.MaritalStatus,
        YearlyIncome  = SOURCE.YearlyIncome, 
        ModifiedDate  = SOURCE.ModifiedDate,
        ETLLastUpdate = CURRENT_TIMESTAMP 
        DELETE WHERE UPPER(SOURCE.ACTIONIND) = 'D'
   WHEN NOT MATCHED 
   THEN  INSERT (CustomerKey, LastName, BirthDate, MaritalStatus, YearlyIncome, ModifiedDate)  
         VALUES (SOURCE.CustomerKey, SOURCE.LastName, SOURCE.BirthDate, 
                 SOURCE.MaritalStatus, SOURCE.YearlyIncome, 
                 SOURCE.ModifiedDate)



#### We need to use a new connection object b/c we need to close it to get the 
#### changes to be committed. 

In [461]:
conn_merge = get_oracle_connection()

In [462]:
with conn_merge.begin() as conn_merge_trans:
    try:
        conn_merge_trans.execute(sql)
    except:
        print('Error processing Merge statement!!!')

In [463]:
pd.read_sql_query("select * from youtube_custtrans", conn)

customerkey lastname  birthdate maritalstatus  yearlyincome actionind  \
0        11000     Yang 1971-10-06             M        250000         U   
1        11001    Jones 1976-05-10             S        360000         U   
2       333301   Murhpy 1975-02-09             M         33000         A   
3       333302     Jain 1980-01-09             M         28000         A   
4        11002   Torres 1971-02-09             M         60000         D   

  modifieddate  
0   2020-01-01  
1   2019-02-01  
2   2018-01-01  
3   2020-02-01  
4   2020-02-01

In [464]:
pd.read_sql_query("SELECT * FROM youtube_customer ORDER BY CustomerKey", conn)

customerkey lastname  birthdate maritalstatus  yearlyincome modifieddate  \
0        11000     Yang 1971-10-06             M        250000   2020-01-01   
1        11001    Jones 1976-05-10             S        360000   2019-02-01   
2        11003      Zhu 1973-08-14             S         70000   2019-01-01   
3       333301   Murhpy 1975-02-09             M         33000   2018-01-01   
4       333302     Jain 1980-01-09             M         28000   2020-02-01   

            etllastupdate  
0 2021-07-04 11:00:56.345  
1 2021-07-04 11:00:56.345  
2 2021-07-04 10:49:43.741  
3 2021-07-04 11:00:56.345  
4 2021-07-04 11:00:56.345

# Let's close the connection. 

In [465]:
conn_merge.dispose()
conn.dispose()

### End of Notebook